<h1><center>
    ECE 438 - Laboratory 8<br/>
    Image Processing Basics: Part II<br/>
    Date:<br/>
    Section:<br/>
</center></h1>

|Name |Signature |Time spent outside lab|
|:---:|:---:|:---:|
|Student Name #1 [---%]| | |
|Student Name #2 [---%]| | |

||Below expectations| Lacks in some respect|Meets all expectations|
|:---:|:---:|:---:|:---:|
|**Completeness of the report**||||
|**Organization of the report**||||
|**Quality of figures**: *Correctly labeled with title, x-axis, y-axis, and name(s)*||||
|**Ability to process images with simple operations (15 pts)**: *Output images, question*||||
|**Understanding of pixel distribution and linear transformation (25 pts)**: *Output images, histograms, code(pointTrans), questions*||||
|**Understanding of gamma correction (15 pts)**: *Output image, code(gammCorr), questions*||||
|**Understanding of image smoothing and sharpening (45 pts)**: *Output images, codes(convolve2d, gaussFilter, medianFilter), frequency response, questions*||||

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt


# refer to https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d.html
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

In [ ]:
# make sure the plot is displayed in this notebook
%matplotlib inline
# specify the size of the plot
plt.rcParams['figure.figsize'] = (16, 6)

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

<h3 style="color:red;"><left>Exercise 1: 2D Convolution</left></h3>

Filters can be represented as a 2-D convolution of an image $f[i, j]$ with the filter’s impulse response $h[i, j]$.

\begin{align*}
    g[i,j]&=f[i,j]**h[i,j]\tag{6}\\
    &=\sum_{k=-\infty}^{\infty}\sum_{l=-\infty}^{\infty}f[i-k,j-l]h[k,l]
\end{align*}

**1. Complete the function below to convolve the image `image` with the filter `kernel`. Use zero padding to make sure the size of the image is unchanged after it being filtered.**

* Assume that the size of the kernel is $s\times s$ where $s$ is odd.
* Try avoid using *for* loops in Equation 6. This can be done by flipping the kernel horizontally and vertically first, and then using element-wise multiplication of matrices and `np.sum()`.
* To zero pad the image, create a new matrix of zeros. The size of this matrix should be $(H+s-1)\times(W+s-1)$, where $H$, $W$ are the height and the width of the original image, and $s$ is the size of the kernel. Then, assign the correct sub-region of the new matrix with the image.

In [ ]:
def convolve2d(image, kernel):
    """
    Parameters
    ---
    image: the input image
    kernel: the filter
    
    Returns
    ---
    filtered: the filtered image
    """
    
    filtered = None
    return filtered

**2. Run the following cell to get the test input image and a $3\times3$ filter. Use your `convolve2d()` function to get the filtered image.**

In [ ]:
image = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                  [0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
                  [0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
                  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]).astype(float)
kernel = np.array([[-1/8, 1/2, -1/8],
                   [-1/4, 1.0, -1/4],
                   [-1/8, 1/2, -1/8]])

In [ ]:
# insert your code here


**3. Run the following cell to check if your `convolve2d()` is correct.**

In [ ]:
filtered_correct = np.array([[0, 0, 0, 0, -1/8, 1/2, -1/8, 0, 0, 0, 0],
                             [0, 0, 0, -1/8, 1/8, 10/8, 1/8, -1/8, 0, 0, 0],
                             [0, 0, -1/8, 1/8, 7/8, 10/8, 7/8, 1/8, -1/8, 0, 0],
                             [0, -1/8, 1/8, 7/8, 9/8, 1, 9/8, 7/8, 1/8, -1/8, 0],
                             [-1/8, 1/8, 7/8, 9/8, 1, 1, 1, 9/8, 7/8, 1/8, -1/8],
                             [-3/8, 1, 9/8, 1, 1, 1, 1, 1, 9/8, 1, -3/8],
                             [-1/2, 3/2, 1, 1, 1, 1, 1, 1, 1, 3/2, -1/2],
                             [-1/2, 3/2, 1, 1, 1, 1, 1, 1, 1, 3/2, -1/2],
                             [-1/2, 3/2, 1, 1, 1, 1, 1, 1, 1, 3/2, -1/2],
                             [-3/8, 9/8, 6/8, 6/8, 6/8, 6/8, 6/8, 6/8, 6/8, 9/8, -3/8],
                             [-1/8, 3/8, 2/8, 2/8, 2/8, 2/8, 2/8, 2/8, 2/8, 3/8, -1/8]])

np.testing.assert_allclose(filtered, filtered_correct, atol=1e-10, rtol=1e-10)

<h3 style="color:red;"><left>Exercise 2: Image Smoothing</left></h3>

**1. Complete the function below that will create a normalized Gaussian filter that is centered around the origin (the center element of your matrix should be $h[0, 0]$).**

* Note that this filter is both separable and symmetric, meaning $h[i, j] = h[i]h[j]$ and $h[i] = h[−i]$.
* Notice that for this filter to be symmetrically centered around zero, `N` will need to be an odd number.
* Make sure the sum of the filter coefficient magnitudes is $1$.

In [ ]:
def gaussFilter(N, var):
    """
    Parameters
    ---
    N: the size of filter
    var: the variance
    
    Returns
    ---
    h: the NxN filter
    """
    
    h = None
    return h

**2. Compute the frequency response of a $7\times7$ Gaussian filter with $\sigma^2=1$.**

* You may use the following command to get a $32\times32$ DFT.
```python
H = np.fft.fftshift(np.fft.fft2(h, (32, 32)))
```

In [ ]:
# insert your code here


**3. Plot the magnitude of the frequency response of the Gaussian filter, $|H_{\text{Gauss}}(\omega_1,\omega_2)|$, using the provided ```mesh_plot()``` function below. Plot it over the region $[-\pi,\pi]\times[-\pi,\pi]$, and label the axes.**

In [ ]:
# refer to https://matplotlib.org/3.1.0/gallery/mplot3d/surface3d.html

def mesh_plot(X, Y, Z, title, xlabel, ylabel):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm, linewidth=2, antialiased=True)
    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))
    # Add a color bar which maps values to colors.
    fig.colorbar(surf, shrink=0.5, aspect=5)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()

In [ ]:
# insert your code here


**4. Load the image file ```race.tif``` and filter it with a $7\times7$ Gaussian filter, with $\sigma^2=1$. Display the original and the filtered images.**

In [ ]:
# insert your code here


**5. Load the image files ```noise1.tif``` and ```noise2.tif```, and display them. These images are versions of `race.tif` that have been degraded by additive white Gaussian noise and “salt and pepper” noise, respectively.**

In [ ]:
# insert your code here


**6. Complete the function below to implement a $3\times3$ median filter (without using the ```signal.medfilt2d()``` function).**

* For convenience, you do not have to alter the pixels on the border of `image`.
* Use ```np.median()``` to find the median value of a subarea of the image, i.e., a $3\times3$ window surrounding each pixel.

In [ ]:
def medianFilter(image):
    """
    Parameters
    ---
    image: the input image
    
    Returns
    ---
    filtered: the output filtered image
    """
    filtered = None
    return filtered

**7. Filter each of the noisy images with both the $7 \times 7$ Gaussian filter ($\sigma^2 = 1$) and the $3 \times 3$ median filter. Display the 4 results and place a title indicating the type of noise and the filter on each image.**

In [ ]:
# insert your code here


**8. Discuss the effectiveness of each filter for the case of additive white Gaussian noise. Discuss both positive and negative effects that you observe for each filter.**

insert your answer here

**9. Discuss the effectiveness of each filter for the case of “salt & pepper” noise. Again, discuss both positive and negative effects that you observe for each filter.**

insert your answer here

<h3 style="color:red;"><left>5.5. Exercise: Image Sharpening</left></h3>

**1. Using your ```gaussFilter()``` function, create a $5 \times 5$ Gaussian filter with $\sigma^2 = 1$.**

In [ ]:
# insert your code here


**2. Derive the frequency response of an unsharp mask filter from equation (10).**

insert your answer here

**3. Compute the frequency response of the unsharp mask filter, using the Gaussian filter as $h[i, j]$, $\alpha = 5$ and $\beta = 4$. The size of the calculated frequency response should be $32 \times 32$. Plot the magnitude of this response in the range $[-\pi, \pi] \times [-\pi, \pi]$ using provided function ```mesh_plot()```, and label the axes. Print out this response.**

In [ ]:
# insert your code here


**4. Load the image file ```blur.tif``` and display it.**

In [ ]:
# insert your code here


**5. Apply the unsharp mask filter with the parameters specified above to this image, using equation (9). Use $\alpha = 5$ and $\beta = 4$ to display the filtered image.**

In [ ]:
# insert your code here


**6. What effect did the filtering have on the image?**

insert your answer here

**7. Now try applying the filter to ```blur.tif```, using $\alpha = 10$ and $\beta = 9$. Label the processed image and display it.**

In [ ]:
# insert your code here


**8. Compare this result to the previous one.**

insert your anwer here

<h3 style="color:red;"><left>Exercise 3: Spatial Filtering</left></h3>


#### Introduction to Spatial Filtering
Spatial filtering is a process used to enhance or restore images by manipulating pixel values in a local neighborhood. In this experiment, we will use spatial filtering to restore images affected by **pepper noise** (random black pixels) and **salt noise** (random white pixels).

One effective filter for this type of noise is the **median filter**. The **median filter** is a type of order-statistic filter that replaces the value of a pixel with the **median** value of its neighboring pixels. This is particularly effective for removing salt-and-pepper noise without significantly blurring the edges of the image.

Another effective approach is using **max and min filters**, which can handle pepper and salt noise individually:
- **Max Filter** is effective in reducing **pepper noise** by taking the maximum value within a local neighborhood, which helps to replace black pixels with brighter surrounding values.
- **Min Filter** is effective in reducing **salt noise** by taking the minimum value within a local neighborhood, which helps to replace white pixels with darker surrounding values.

#### Principle of Spatial Filtering
To remove salt or pepper noise, we use spatial filters that smooth the image by averaging or taking other statistical measures of neighboring pixel values.

- **Median Filter**:
  $$ f_{\text{median}}(x, y) = \text{median} \{ g(r, c) \}, \quad (r, c) \in S_{xy} $$
  Where $ S_{xy} $ represents the neighborhood around the pixel at position $ (x, y) $, and $ g(r, c) $ are the intensity values of pixels within the neighborhood. The median filter is particularly effective for removing salt or pepper noise without significantly blurring the edges of the image.

- **Max Filter**:
  $$ f_{\text{max}}(x, y) = \max \{ g(r, c) \}, \quad (r, c) \in S_{xy} $$
  The max filter replaces each pixel value with the maximum value from its neighborhood. This is useful for removing pepper noise.

- **Min Filter**:
  $$ f_{\text{min}}(x, y) = \min \{ g(r, c) \}, \quad (r, c) \in S_{xy} $$
  The min filter replaces each pixel value with the minimum value from its neighborhood. This is useful for removing salt noise.

#### Student Task
1. **Apply spatial filters** (max filter and min filter) to given noisy images that are affected by **pepper noise** and **salt noise** separately.
2. Compare the results of the filters to determine which one works better for removing each type of noise while preserving details.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to apply max filter
def max_filter(image, kernel_size=3):
    # insert your code here. Hint: check cv2.dilate()
    return None


# Function to apply min filter
def min_filter(image, kernel_size=3):
    # insert your code here. Hint: check cv2.erode()
    return None

# Read and filter the images
pepper_noise_image = cv2.imread('circuitboard-pepper.tif', cv2.IMREAD_GRAYSCALE)
max_filtered_pepper = max_filter(pepper_noise_image, kernel_size=3)

salt_noise_image = cv2.imread('circuitboard-salt.tif', cv2.IMREAD_GRAYSCALE)
min_filtered_salt = min_filter(salt_noise_image, kernel_size=3)

# Display the images before and after filtering
plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Pepper Noise Image')
plt.axis('off')
plt.imshow(pepper_noise_image, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Max Filtered (Pepper Noise)')
plt.axis('off')
plt.imshow(max_filtered_pepper, cmap='gray')
plt.show()

plt.figure(figsize=(8, 4))
plt.subplot(1, 2, 1)
plt.title('Salt Noise Image')
plt.axis('off')
plt.imshow(salt_noise_image, cmap='gray')

plt.subplot(1, 2, 2)
plt.title('Min Filtered (Salt Noise)')
plt.imshow(min_filtered_salt, cmap='gray')
plt.axis('off')
plt.show()

<h3 style="color:red;"><left>Exercise 4: Adaptive Median Filtering</left></h3>



#### Introduction to Adaptive Median Filtering
The filters discussed previously, such as median, max, and min filters, are effective but do not adapt to the varying characteristics of the image. In some cases, using an **adaptive median filter** can improve results by adjusting the filtering based on local image characteristics. The **adaptive median filter** works by changing the size of the neighborhood used for filtering until specific conditions are met. This makes it effective for images with high-density salt-and-pepper noise.

#### Principle of Adaptive Median Filtering
In adaptive median filtering, the process starts with a small neighborhood size $ S_{xy} $ centered at the pixel being processed. The algorithm uses two levels of processing for each pixel to decide whether it should be replaced or retained. The conditions are as follows:

- **Level A**:
  - If $ z_{\text{min}} < z_{\text{med}} < z_{\text{max}} $, go to **Level B**.
  - Else, increase the size of $ S_{xy} $.
    - If $ S_{xy} \leq S_{\text{max}} $, repeat **Level A**.
    - Else, output $ z_{\text{med}} $.
- **Level B**:
  - If $ z_{\text{min}} < z_{xy} < z_{\text{max}} $, output $ z_{xy} $.
  - Else, output $ z_{\text{med}} $.

Where:
- $ z_{\text{min}} $: Minimum intensity value in $ S_{xy} $.
- $ z_{\text{max}} $: Maximum intensity value in $ S_{xy} $.
- $ z_{\text{med}} $: Median of intensity values in $ S_{xy} $.
- $ z_{xy} $: Intensity value at coordinates $(x, y)$.
- $ S_{\text{max}} $: Maximum allowed size of $ S_{xy} $.

#### Student Task
1. **Add salt-and-pepper noise** to a given image with salt-and-pepper noise (probability = 0.25).
2. **Apply adaptive median filtering** to the noisy image.
3. Compare the result of adaptive median filtering with traditional median filtering to determine which one works better for high-density noise while preserving details.


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Function to add salt-and-pepper noise to an image
def add_salt_pepper_noise(image, prob=0.25):
    noisy_image = image.copy()
    num_salt = np.ceil(prob * image.size * 0.5)
    num_pepper = np.ceil(prob * image.size * 0.5)
    
    salt_coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]    
    pepper_coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    
    # insert your code here to add salt and pepper noise to the images according to the coordinates
    noisy_image = None
    
    return noisy_image

# Function to apply adaptive median filter
def adaptive_median_filter(image, S_max=7):
    output_image = image.copy()
    rows, cols = image.shape
    
    for i in range(rows):
        for j in range(cols):
            S_xy = 3
            while S_xy <= S_max:
                r_min = max(i - S_xy // 2, 0)
                r_max = min(i + S_xy // 2 + 1, rows)
                c_min = max(j - S_xy // 2, 0)
                c_max = min(j + S_xy // 2 + 1, cols)
                
                neighborhood = image[r_min:r_max, c_min:c_max]
                z_xy = image[i, j]
                
                # insert your code here. Hint: Calculate z_min, z_max, and z_med using neighborhood values
                z_min = None
                z_max = None
                z_med = None
                
                # insert your code here. Hint: assign correct values for output_image according to the above instructions
                if z_min < z_med < z_max:
                    if z_min < z_xy < z_max:
                        output_image[i, j] = None
                    else:
                        output_image[i, j] = None
                    break
                else:
                    S_xy += 2
                    if S_xy > S_max:
                        output_image[i, j] = None
    
    return output_image

def median_filter(image, kernel_size=3):
    # insert your code here. Hint: check cv2.medianBlur().
    return None

original_image = cv2.imread('girl.tif', cv2.IMREAD_GRAYSCALE)
salt_pepper_noise_image = add_salt_pepper_noise(original_image, prob=0.25)
adaptive_filtered_image = adaptive_median_filter(salt_pepper_noise_image, S_max=7)
traditional_median_filtered_image = median_filter(salt_pepper_noise_image, 3)

# Display original, noisy, and filtered images
plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.title('Original Image')
plt.axis('off')
plt.imshow(original_image, cmap='gray')

plt.subplot(1, 4, 2)
plt.title('Noisy Image (Salt & Pepper, p=0.25)')
plt.axis('off')
plt.imshow(salt_pepper_noise_image, cmap='gray')

plt.subplot(1, 4, 3)
plt.title('Adaptive Median Filtered Image')
plt.axis('off')
plt.imshow(adaptive_filtered_image, cmap='gray')

plt.subplot(1, 4, 4)
plt.title('Traditional Median Filtered Image')
plt.axis('off')
plt.imshow(traditional_median_filtered_image, cmap='gray')
plt.show()

<h3 style="color:red;"><left>Exercise 5: MSE and PSNR</left></h3>



#### Introduction to Image Quality Evaluation
In image processing, it’s important to assess how well filtering methods retain the quality of the original image while removing noise. Two commonly used metrics for this purpose are **Mean Squared Error (MSE)** and **Peak Signal-to-Noise Ratio (PSNR)**.

- **Mean Squared Error (MSE)**: MSE measures the average squared difference between the original and filtered images. A lower MSE value indicates a closer match to the original image, meaning the filter was effective in maintaining image quality.
  
  $$
  \text{MSE} = \frac{1}{mn} \sum_{x=1}^{m} \sum_{y=1}^{n} [I_{\text{original}}(x, y) - I_{\text{filtered}}(x, y)]^2
  $$

  where \( m \) and \( n \) are the image dimensions, \( I_{\text{original}} \) is the pixel value in the original image, and \( I_{\text{filtered}} \) is the corresponding pixel value in the filtered image.

- **Peak Signal-to-Noise Ratio (PSNR)**: PSNR, often expressed in decibels (dB), is based on the MSE but provides a standardized scale to evaluate image quality. A higher PSNR value indicates better image quality and less distortion from the original.

  $$
  \text{PSNR} = 20 \cdot \log_{10} \left( \frac{\text{MAX}_{I}}{\sqrt{\text{MSE}}} \right)
  $$

  where \( \text{MAX}_{I} \) is the maximum possible pixel value in the image (typically 255 for an 8-bit grayscale image).

#### Principle of Quality Metrics
- **MSE** helps us understand the average discrepancy between the original and filtered images. A low MSE value indicates high similarity, which is desirable for preserving image quality.
- **PSNR** gives us a standardized measure of quality; the higher the PSNR, the better the filter retained details while reducing noise.

#### Student Task
1. **Implement MSE and PSNR**: Write code to compute MSE and PSNR between the original and filtered images.
2. **Apply Evaluation Metrics**: Calculate MSE and PSNR for each filter you implemented (max filter, min filter, adaptive median filter, and traditional median filter).
3. **Analyze the Results**: Based on the calculated MSE and PSNR values, analyze and compare the performance of each filter. Which filter achieved the lowest MSE and highest PSNR, indicating effective noise removal with minimal loss of detail?


In [ ]:
import cv2
import numpy as np

# Function to calculate Mean Squared Error (MSE)
def calculate_mse(original_image, filtered_image):
    # insert your code here. Hint: use numpy matrix calculation for the above formula.
    mse = None
    return mse

# Function to calculate Peak Signal-to-Noise Ratio (PSNR)
def calculate_psnr(original_image, filtered_image):
    # insert your code here. Hint: return float('inf') if MSE equals to 0.
    psnr = None
    return psnr

# Calculate MSE and PSNR for each filtered image

original_image = cv2.imread('house.tif', cv2.IMREAD_GRAYSCALE)
salt_pepper_noise_image = add_salt_pepper_noise(original_image, prob=0.25)

max_filtered_image = max_filter(salt_pepper_noise_image, kernel_size=3)
min_filtered_image = min_filter(salt_pepper_noise_image, kernel_size=3)
adaptive_filtered_image = adaptive_median_filter(salt_pepper_noise_image, S_max=7)
traditional_median_filtered_image = median_filter(salt_pepper_noise_image, 3)

mse_max_filter = calculate_mse(original_image, max_filtered_image)
psnr_max_filter = calculate_psnr(original_image, max_filtered_image)

mse_min_filter = calculate_mse(original_image, min_filtered_image)
psnr_min_filter = calculate_psnr(original_image, min_filtered_image)

mse_adaptive_median = calculate_mse(original_image, adaptive_filtered_image)
psnr_adaptive_median = calculate_psnr(original_image, adaptive_filtered_image)

mse_traditional_median = calculate_mse(original_image, traditional_median_filtered_image)
psnr_traditional_median = calculate_psnr(original_image, traditional_median_filtered_image)


# Display original, noisy, and filtered images
plt.figure(figsize=(12, 5))
plt.subplot(2, 3, 1)
plt.title('Original Image')
plt.axis('off')
plt.imshow(original_image, cmap='gray')

plt.subplot(2, 3, 2)
plt.title('Noisy Image (Salt & Pepper, p=0.25)')
plt.axis('off')
plt.imshow(salt_pepper_noise_image, cmap='gray')

plt.subplot(2, 3, 3)
plt.title('Max Filtered Image')
plt.axis('off')
plt.imshow(max_filtered_image, cmap='gray')

plt.subplot(2, 3, 4)
plt.title('Min Filtered Image')
plt.axis('off')
plt.imshow(min_filtered_image, cmap='gray')

plt.subplot(2, 3, 5)
plt.title('Adaptive Median Filtered Image')
plt.axis('off')
plt.imshow(adaptive_filtered_image, cmap='gray')

plt.subplot(2, 3, 6)
plt.title('Traditional Median Filtered Image')
plt.axis('off')
plt.imshow(traditional_median_filtered_image, cmap='gray')

plt.show()




In [ ]:
# Print MSE and PSNR values for comparison
print(f"Max Filter - MSE: {mse_max_filter:.2f}, PSNR: {psnr_max_filter:.2f}")
print(f"Min Filter - MSE: {mse_min_filter:.2f}, PSNR: {psnr_min_filter:.2f}")
print(f"Adaptive Median Filter - MSE: {mse_adaptive_median:.2f}, PSNR: {psnr_adaptive_median:.2f}")
print(f"Traditional Median Filter - MSE: {mse_traditional_median:.2f}, PSNR: {psnr_traditional_median:.2f}")